# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Data importeren

In [2]:
df = pd.read_csv('seriousgaming.csv', sep=';')
df.head

<bound method NDFrame.head of     Id Gender  Leeftijd Karakter Apparaat Huis Plek  \
0    1      v        19        b        a    b    b   
1    2      v        22        a        a    b    a   
2    3      v        20        a        a    a    a   
3    4      m        24        b        a    b    b   
4    5      v        19        b        a    b    b   
..  ..    ...       ...      ...      ...  ...  ...   
63  65      m        25        a        a    b    b   
64  66      m        61        b        a    b    b   
65  67      v        29        a        a    b    a   
66  68      v        55        b        a    b    b   
67  69      m        18        b        a    b    b   

   Rugzak [A: In kijken (-1); Laten liggen (0); Melding maken (1)]  \
0                                                   b                
1                                                   a                
2                                                   a                
3                            

# Data cleanen

In [3]:
df = df.rename(columns={'Veiligheidsbewust [0: Nee; 1:Ja]': 'Veiligheidsbewust'})
df = df[['Karakter', 'Apparaat', 'Huis', 'Plek', 'Uitslag', 'Veiligheidsbewust']]
df.head()

,Karakter,Apparaat,Huis,Plek,Uitslag,Veiligheidsbewust
0,b,a,b,b,0,0
1,a,a,b,a,-3,0
2,a,a,a,a,-3,0
3,b,a,b,b,1,1
4,b,a,b,b,0,1


Aantal veiligheidsbewuste mensen:

In [4]:
df['Veiligheidsbewust'].value_counts()

1    41
0    27
Name: Veiligheidsbewust, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Karakter           68 non-null     object
 1   Apparaat           68 non-null     object
 2   Huis               68 non-null     object
 3   Plek               68 non-null     object
 4   Uitslag            68 non-null     int64 
 5   Veiligheidsbewust  68 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 3.3+ KB


### Dummificeren
We willen dummy variabelen maken voor *Karakter*, *Apparaat*, *Huis*, *Plek*.

In [6]:
df = pd.get_dummies(df, columns=['Karakter', 'Apparaat', 'Huis', 'Plek'], drop_first = False)

# Swap Uitslag & Veiligheidsbewust
titles = list(df.columns)
titles[0], titles[1] = titles[1], titles[0]
df = df[titles]

df.head()

,Veiligheidsbewust,Uitslag,Karakter_a,Karakter_b,Apparaat_a,Apparaat_b,Huis_a,Huis_b,Plek_a,Plek_b
0,0,0,0,1,1,0,0,1,0,1
1,0,-3,1,0,1,0,0,1,1,0
2,0,-3,1,0,1,0,1,0,1,0
3,1,1,0,1,1,0,0,1,0,1
4,1,0,0,1,1,0,0,1,0,1


X pakken en data splitten

In [7]:
X = df.loc[:, 'Uitslag' : 'Plek_b']
y = df['Veiligheidsbewust'] # Dit willen we weten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Trainen algoritme

In [8]:
rf = RandomForestClassifier(random_state=1, n_estimators=100) # RF is a random algorithm, so to get the same results we need to use random_state
rf = rf.fit(X_train, y_train)

Accuracy =

In [9]:
rf.score(X_test,y_test)

0.9047619047619048

In [10]:
rf.classes_

array([0, 1], dtype=int64)

In [11]:
y_pred = rf.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_pred) #creates a "confusion matrix"
cm = pd.DataFrame(cm, index=['Niet veiligheidsbewust (actual)', 'Wel veiligheidsbewust (actual)'], columns = ['Niet veiligheidsbewust (pred)', 'Wel veiligheidsbewust (pred)']) #label and make df
cm

,Niet veiligheidsbewust (pred),Wel veiligheidsbewust (pred)
Niet veiligheidsbewust (actual),7,2
Wel veiligheidsbewust (actual),0,12


| TP | FP |
|---|---|
| FN | TN |

**True Positive**: 7x verwachten we **niet** veiligheidsbewust --> en is ook **niet** veiligheidsbewust <br/>
**True Negative**: 12x verwachten we **wel** veiligheidsbewust --> en is ook **wel** veiligheidsbewust<br/>
**False Positive**: 2x verwachten we **wel** veiligheidsbewust --> maar is **niet** veiligheidsbewust <br/>
**False Negative**: 0x verwachten we **niet** veiligheidsbewust --> maar is **wel** veiligheidsbewust <br/>

In [12]:
print("Specificity (TN) =", 12 / (12 + 2))

Specificity (TN) = 0.8571428571428571


# Voorspellen

Vul de vragenlijst in

In [13]:
optie1, optie2, optie3, optie4, object1, object2, object3 = "", "", "", "", "", "", ""

while optie1 != 'a' and optie1 != 'b':
    optie1 = str(input("1) Karakter A of B: "))
while optie2 != 'a' and optie2 != 'b':
    optie2 = str(input("2) Apparaat A of B: "))
while optie3 != 'a' and optie3 != 'b':
    optie3 = str(input("3) Huis A of B: "))
while optie4 != 'a' and optie4 != 'b':
    optie4 = str(input("4) Plek A of B: "))
#----------------------------------
while object1 != 'a' and object1 != 'b' and object1 != 'c':
    object1 = str(input("5) Rugzak: "))
while object2 != 'a' and object2 != 'b' and object2 != 'c':
    object2 = str(input("6) Koffer: "))
while object3 != 'a' and object3 != 'b' and object3 != 'c':
    object3 = str(input("7) Envelop: "))

objecten_score = 0
if object1 == 'a':
    objecten_score-= 1
elif object1 == 'c':
    objecten_score+= 1

if object2 == 'a':
    objecten_score-= 1
elif object2 == 'c':
    objecten_score+= 1

if object3 == 'a':
    objecten_score-= 1
elif object3 == 'c':
    objecten_score+= 1


print(objecten_score)

data = [[objecten_score, optie1, optie2, optie3, optie4]] 

-3


In [14]:
#data = [[1, 'a', 'b', 'a', 'b']]
df2 = pd.DataFrame(data, columns = ['Uitslag', 'Karakter', 'Apparaat', 'Huis', 'Plek'])
print(df2)

df2 = pd.get_dummies(df2, columns=['Karakter', 'Apparaat', 'Huis', 'Plek'], drop_first=False)

test = pd.DataFrame(df2, columns=['Uitslag', 'Karakter_a', 'Karakter_b', 'Apparaat_a', 'Apparaat_b', 'Huis_a', 'Huis_b', 'Plek_a', 'Plek_b'])
test = test.fillna(0)
test = test.astype(int)

pred = rf.predict(test)
print("Prediction: ", pred)


   Uitslag Karakter Apparaat Huis Plek
0       -3        a        b    a    b
Prediction:  [0]


# Compacte Dashboard

In [15]:
from termcolor import colored

if pred == 1:
    print (colored('Deze persoon is'), colored('veiligheidsbewust', 'green'))
else:
    print (colored('Deze persoon is'), colored('niet veiligheidsbewust', 'red'))

print()

print("Antwoorden keuzes:")
print("--------------------------------------")
print("Karakter:", optie1)
print("Opmerkingen: -")
print("Apparaat:", optie2)
print("Opmerkingen: -")
print("Huis:", optie3)
print("Opmerkingen: -")
print("Locatie:", optie4)
print("Opmerkingen: -")

print()

print("Gameobjecten:")
print("--------------------------------------")
if object1 == 'a':
    print("Koffer in de trein:", colored(object1, 'red'))
elif object1 == 'b':
    print("Koffer in de trein:", colored(object1, 'yellow'))
elif object1 == 'c':
    print("Koffer in de trein:", colored(object1, 'green'))
print("Opmerkingen: -")

if object2 == 'a':
    print("Rugtas in winkelcentrum:", colored(object2, 'red'))
elif object2 == 'b':
    print("Rugtas in winkelcentrum:", colored(object2, 'yellow'))
elif object2 == 'c':
    print("Rugtas in winkelcentrum:", colored(object2, 'green'))
print("Opmerkingen: -")

if object3 == 'a':
    print("Envelop in de bibliotheek:", colored(object3, 'red'))
elif object3 == 'b':
    print("Envelop in de bibliotheek:", colored(object3, 'yellow'))
elif object3 == 'c':
    print("Envelop in de bibliotheek:", colored(object3, 'green'))
print("Opmerkingen: -")

Deze persoon is niet veiligheidsbewust

Antwoorden keuzes:
--------------------------------------
Karakter: a
Opmerkingen: -
Apparaat: b
Opmerkingen: -
Huis: a
Opmerkingen: -
Locatie: b
Opmerkingen: -

Gameobjecten:
--------------------------------------
Koffer in de trein: a
Opmerkingen: -
Rugtas in winkelcentrum: a
Opmerkingen: -
Envelop in de bibliotheek: a
Opmerkingen: -
